In [ ]:
import xarray as xr

In [ ]:
ds_of800_clim = xr.open_dataset('/cluster/projects/nn9297k/OF800/data_clim/OF800_his_merged.nc')
ds_of160_clim_salt = xr.open_dataset('/cluster/projects/nn9297k/OF160/Clm/098_OF160_clm_salt.nc')
ds_of800_bry = xr.open_dataset('/cluster/projects/nn9297k/OF800/data_bry/OF800_bry.nc')

In [ ]:
ds_of800_bry

In [ ]:
da_salt_west = ds_of160_clim_salt.salt.isel(xi_rho=0)
da_salt_east = ds_of160_clim_salt.salt.isel(xi_rho=-1)
da_salt_south = ds_of160_clim_salt.salt.isel(eta_rho=0)
da_salt_north = ds_of160_clim_salt.salt.isel(eta_rho=-1)

In [ ]:
da_salt_south

In [ ]:
da_salt_west.values.shape

In [ ]:
ds = xr.Dataset({
    "salt_west": (["eta_rho", "ocean_time", "s_rho"], da_salt_west.values),
    "salt_east": (["eta_rho", "ocean_time", "s_rho"], da_salt_east.values),
    "salt_south": (["xi_rho", "ocean_time", "s_rho"], da_salt_south.values),
    "salt_north": (["xi_rho", "ocean_time", "s_rho"], da_salt_north.values),
    })

In [ ]:
ds